In [2]:
!pip3 install rasterio

     |████████████████████████████████| 18.2MB 168kB/s ta 0:00:01   |████████████████████████▏       | 13.8MB 24kB/s eta 0:02:58


In [14]:
import numpy as np
import os

import rasterio
from rasterio import Affine
from rasterio.windows import Window
from rasterio.transform import from_bounds

from pathlib import Path

import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm

from skimage.transform import rescale, resize

from fastai.vision import *

In [4]:
def get_windows(rst_h, rst_w, max_h, max_w, col_off = 0, row_off = 0):
    wins = []
    rows = rst_h // max_h + 1
    cols = rst_w // max_w + 1    
    
    for r in range(rows):
        if r == rows-1: height = rst_h % max_h
        else: height = max_h
            
        for c in range(cols):
            if c == cols-1: width = rst_w % max_w
            else: width = max_w

            if width != 0 and height != 0: 
                wins.append(((r,c),Window(c*max_w+col_off, r*max_h+row_off, width, height)))
    return wins

def get_tfm(window, rst_full):
    c_o, r_o, w, h = window.flatten()
    left, top, right, bottom = *rst_full.xy(r_o, c_o, offset='ul'), *rst_full.xy(r_o+h, c_o+w, offset='lr')
    tfm = from_bounds(left,bottom,right,top, w, h)
    return tfm

def save_subwin(arr, crs, tfm, save_fn):
    im = (arr*255).astype('uint8')
    with rasterio.open(OUTPUT/f'{save_fn}.tif', 'w', driver='GTiff', 
                            height=im.shape[0], width=im.shape[1],
                            count=3, dtype=im.dtype, crs=crs, transform=tfm, compress='JPEG', tiled=True) as dst:
        for k, a in [(1, im), (2, im), (3, im)]:
            dst.write(a, indexes=k)
            
def pad_window(window, pad):
    col_off, row_off, width, height = window.flatten()
    return Window(col_off-pad//2, row_off-pad//2,width+pad,height+pad)

In [35]:
COG_URL = Path('/content/drive/My Drive/beijing_aerial/2008.tif')
WORKDIR = Path('/content/drive/My Drive/beijing_aerial/segout')
WORKDIR_sh = Path('/content/drive/My\ Drive/beijing_aerial/segout')
OUTPUT = WORKDIR/'2008/outputs'
path_lbl = WORKDIR/'2008/labels'
path_img = WORKDIR/'2008/images'
list(map(lambda path: os.makedirs(path, exist_ok=True), [WORKDIR, OUTPUT, path_lbl, path_img]))

raster = rasterio.open(COG_URL,'r')
raster.meta

{'count': 3,
 'crs': CRS.from_wkt('PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",6378137,0]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]],PROJECTION["Mercator_2SP"],PARAMETER["standard_parallel_1",0],PARAMETER["central_meridian",0],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]'),
 'driver': 'GTiff',
 'dtype': 'uint8',
 'height': 12062,
 'nodata': None,
 'transform': Affine(1.0871161111356706, 0.0, 12950476.133884454,
       0.0, -1.0871161111356706, 4859280.992473818),
 'width': 11244}

In [ ]:
get_y_fn = lambda x: path_lbl/f'{x.stem}{x.suffix}'
codes = np.array(['Empty','Building'])
unet_sz = 224
bs=1
data = (SegmentationItemList.from_folder(path_img)
        .random_split_by_pct()
        .label_from_func(get_y_fn, classes=codes)
        .transform(get_transforms(), tfm_y=True, size=unet_sz)
        .databunch(bs=bs)
        .normalize(imagenet_stats))        

learn = unet_learner(data, models.resnet34)
learn.load(WORKDIR/'dice80.pth')
learn.model.eval()

In [38]:
! cp $WORKDIR_sh/export.pkl .
learn = load_learner(Path('.'),'export.pkl')

cp: error reading '/content/drive/My Drive/beijing_aerial/segout/export.pkl': Input/output error


EOFError: Ran out of input

In [37]:
learn

NameError: name 'learn' is not defined